In [10]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-12-01 04:48:21--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2025-12-01 04:48:21 (19.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [45]:
# read it in to inspect it
with open('/content/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [46]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [47]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [48]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [49]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [50]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [51]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [52]:
X = train_data[:block_size]
y = train_data[1: block_size + 1]
for i in range(block_size):
  context = X[:i+1]
  target = y[i]
  print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [53]:
# Apply with Pytorch
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  print(ix)
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when input is {context.tolist()} the target: {target}")

tensor([ 76049, 234249, 934904, 560986])
inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1,

In [55]:
xb

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

In [73]:
import math
import torch
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    self.vocab_size = vocab_size # Store vocab_size for use in forward

  def forward(self, idx, targets=None):
    logits = self.token_embedding_table(idx) # (Batch, Time, Channel)
    if targets is not None:
      B, T, C = logits.shape

      # print(B, T, C)
      logits = logits.view(B*T, C)
      # print(logits)
      # print(targets.shape)
      targets = targets.view(B*T)
      # print(targets.shape)

      loss = F.cross_entropy(logits, targets)
    else:
      loss = None

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self(idx)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)
    return idx

model = BigramLanguageModel(vocab_size)
out, loss = model(xb, yb)
print('minimum loss: ', -math.log(1/vocab_size))
print(out)
print(loss)


idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))

minimum loss:  4.174387269895637
tensor([[-0.1040, -0.5675, -0.1404,  ...,  1.3076, -0.5611, -1.2666],
        [ 2.1952,  0.5946, -1.2645,  ..., -0.6410, -2.0446, -0.3275],
        [ 1.7859, -0.2828,  1.1437,  ..., -0.0643,  0.5074,  0.0902],
        ...,
        [ 0.1686,  0.5584, -0.8691,  ...,  0.3273, -0.1228,  0.8749],
        [ 2.1952,  0.5946, -1.2645,  ..., -0.6410, -2.0446, -0.3275],
        [ 1.8182,  1.7594, -1.1061,  ..., -0.5361,  1.2962,  0.5031]],
       grad_fn=<ViewBackward0>)
tensor(4.5084, grad_fn=<NllLossBackward0>)

DTDVwDwbMMvCo,XNk!BrU&-'HJrm'lMkibjLFeI&;UGf'O OE.?LdAsW$Mr?MLCRJo,lo-EnsXBVRDbq3f$n3f;xgWlHsBJ?B GT


tensor([[[[0]]]])

In [71]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [75]:
epochs = 1000
for steps in range(epochs):
  xb, yb = get_batch('train')

  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

tensor([819247, 261500, 111034, 542125])
tensor([178209, 414324, 810854, 672668])
tensor([962563, 172013, 685607, 240112])
tensor([ 14312, 439796,  87134, 378560])
tensor([365670, 338607, 135372, 991402])
tensor([856154, 332749, 294576, 788114])
tensor([ 23159, 617795, 614531, 488622])
tensor([713592, 288531, 709717, 190916])
tensor([ 29894, 312927, 741147, 227019])
tensor([687457, 936772, 637695,  89209])
tensor([607460, 450059, 717350,  28025])
tensor([116781, 615321, 803435, 348039])
tensor([1000368,  494239,  673028,  312278])
tensor([281914, 745435, 465708, 161923])
tensor([130408, 448857, 174051, 574053])
tensor([579855,  90638, 409909, 186925])
tensor([493892, 957437, 758109, 105756])
tensor([ 27966, 710542, 749699, 522769])
tensor([629679,   2748, 918554,  28824])
tensor([394488, 420626, 849519,  45049])
tensor([    71, 157042, 449558, 672442])
tensor([155657, 713146, 817077,   2322])
tensor([ 66752, 412344, 973492, 862863])
tensor([883642, 252000, 558974, 439706])
tensor([7024

In [94]:
context_str = 'save means to'

context_str_ids = encode(context_str)
context_str_ids = torch.tensor(context_str_ids, dtype=torch.long)
print(context_str_ids)

tensor([57, 39, 60, 43,  1, 51, 43, 39, 52, 57,  1, 58, 53])


In [95]:
idx = context_str_ids.unsqueeze(0)
idx

tensor([[57, 39, 60, 43,  1, 51, 43, 39, 52, 57,  1, 58, 53]])

In [97]:
generated_ids = decode(model.generate(idx, max_new_tokens=10)[0].tolist())

generated_ids

'save means ton&Bt\nyhXTI'

## The mathematical trick in self attention

In [99]:
torch.manual_seed(1337)
B, T, C  = 4, 8, 2
x = torch.rand(B, T, C)
x

tensor([[[0.0783, 0.4956],
         [0.6231, 0.4224],
         [0.2004, 0.0287],
         [0.5851, 0.6967],
         [0.1761, 0.2595],
         [0.7086, 0.5809],
         [0.0574, 0.7669],
         [0.8778, 0.2434]],

        [[0.6005, 0.7079],
         [0.5102, 0.4065],
         [0.8864, 0.3578],
         [0.8963, 0.9639],
         [0.0148, 0.7358],
         [0.7884, 0.9469],
         [0.4656, 0.8378],
         [0.1269, 0.2132]],

        [[0.2944, 0.3677],
         [0.4830, 0.6752],
         [0.5226, 0.5536],
         [0.5701, 0.2481],
         [0.6040, 0.7077],
         [0.0790, 0.7629],
         [0.7708, 0.0743],
         [0.5760, 0.6853]],

        [[0.9100, 0.7684],
         [0.7136, 0.0586],
         [0.7642, 0.9663],
         [0.9940, 0.7994],
         [0.1150, 0.6722],
         [0.0869, 0.6041],
         [0.4242, 0.1002],
         [0.0840, 0.5361]]])

In [105]:
# We want x[b, t] = mean_{i <= t} x[b, i]

xbow = torch.zeros((B, T, C))
count = 0
for b in range(B):
  for t in range(T):
    print(b, t)
    xprev = x[b, :t+1]
    print(xprev)
    xbow[b, t] = torch.mean(xprev, 0)

    count += 1
    if count == 10:
      break


0 0
tensor([[0.0783, 0.4956]])
0 1
tensor([[0.0783, 0.4956],
        [0.6231, 0.4224]])
0 2
tensor([[0.0783, 0.4956],
        [0.6231, 0.4224],
        [0.2004, 0.0287]])
0 3
tensor([[0.0783, 0.4956],
        [0.6231, 0.4224],
        [0.2004, 0.0287],
        [0.5851, 0.6967]])
0 4
tensor([[0.0783, 0.4956],
        [0.6231, 0.4224],
        [0.2004, 0.0287],
        [0.5851, 0.6967],
        [0.1761, 0.2595]])
0 5
tensor([[0.0783, 0.4956],
        [0.6231, 0.4224],
        [0.2004, 0.0287],
        [0.5851, 0.6967],
        [0.1761, 0.2595],
        [0.7086, 0.5809]])
0 6
tensor([[0.0783, 0.4956],
        [0.6231, 0.4224],
        [0.2004, 0.0287],
        [0.5851, 0.6967],
        [0.1761, 0.2595],
        [0.7086, 0.5809],
        [0.0574, 0.7669]])
0 7
tensor([[0.0783, 0.4956],
        [0.6231, 0.4224],
        [0.2004, 0.0287],
        [0.5851, 0.6967],
        [0.1761, 0.2595],
        [0.7086, 0.5809],
        [0.0574, 0.7669],
        [0.8778, 0.2434]])
1 0
tensor([[0.6005, 0.7

In [101]:
xbow

tensor([[[0.0783, 0.4956],
         [0.3507, 0.4590],
         [0.3006, 0.3156],
         [0.3717, 0.4108],
         [0.3326, 0.3806],
         [0.3953, 0.4140],
         [0.3470, 0.4644],
         [0.4134, 0.4368]],

        [[0.6005, 0.7079],
         [0.5554, 0.5572],
         [0.6657, 0.4908],
         [0.7234, 0.6090],
         [0.5817, 0.6344],
         [0.6161, 0.6865],
         [0.5946, 0.7081],
         [0.5362, 0.6462]],

        [[0.2944, 0.3677],
         [0.3887, 0.5215],
         [0.4333, 0.5322],
         [0.4675, 0.4611],
         [0.4948, 0.5105],
         [0.4255, 0.5525],
         [0.4748, 0.4842],
         [0.4875, 0.5094]],

        [[0.9100, 0.7684],
         [0.8118, 0.4135],
         [0.7959, 0.5978],
         [0.8454, 0.6482],
         [0.6993, 0.6530],
         [0.5973, 0.6449],
         [0.5726, 0.5670],
         [0.5115, 0.5632]]])

In [106]:
import torch

torch.mean(torch.tensor([[0.0783, 0.4956],
        [0.6231, 0.4224],
        [0.2004, 0.0287]]))

tensor(0.3081)